# Sparse Autoencoder Training Demo

## Setup

In [12]:
# Autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from sparse_autoencoder import TensorActivationStore, SparseAutoencoder, pipeline
from sparse_autoencoder.source_data.pile_uncopyrighted import PileUncopyrightedDataset
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
from transformers import PreTrainedTokenizerBase
import torch
import wandb

In [14]:
device = get_device()

### Source Model

In [15]:
src_model = HookedTransformer.from_pretrained("solu-1l", dtype="float32")
src_d_mlp: int = src_model.cfg.d_mlp  # type: ignore
src_d_mlp

Loaded pretrained model solu-1l into HookedTransformer


2048

### Source Dataset

In [16]:
tokenizer: PreTrainedTokenizerBase = src_model.tokenizer  # type: ignore
source_data = PileUncopyrightedDataset(tokenizer=tokenizer)
src_dataloader = source_data.get_dataloader(batch_size=8)

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

### Activation Store

In [17]:
max_items = 1_000_000
store = TensorActivationStore(max_items, src_d_mlp, device)

### Autoencoder

In [18]:
autoencoder = SparseAutoencoder(src_d_mlp, src_d_mlp * 8, torch.zeros(src_d_mlp))
autoencoder

SparseAutoencoder(
  (encoder): Sequential(
    (0): TiedBias(position=TiedBiasPosition.PRE_ENCODER)
    (1): ConstrainedUnitNormLinear(in_features=2048, out_features=16384, bias=True)
    (2): ReLU()
  )
  (decoder): Sequential(
    (0): ConstrainedUnitNormLinear(in_features=16384, out_features=2048, bias=False)
    (1): TiedBias(position=TiedBiasPosition.POST_DECODER)
  )
)

## Training

If you initialise [wandb](https://wandb.ai/site), the pipeline will automatically log all metrics to wandb.

In [19]:
# wandb.init(project="sparse-autoencoder", dir=".cache/wandb")

In [20]:
pipeline(
    src_model=src_model,
    src_model_activation_hook_point="blocks.0.mlp.hook_post",
    src_model_activation_layer=0,
    src_dataloader=src_dataloader,
    activation_store=store,
    num_activations_before_training=max_items,
    autoencoder=autoencoder,
    device=device,
)

Generate/Train Cycles: 0it [00:00, ?it/s]

Generate Activations:   0%|          | 0/2000000 [00:00<?, ?it/s]

: 